# Preprocess the authors

This notebook aims to preprocess the author data and save to shelve database,
as well as testing the read write functions

## 1. Import libraries and set up.

In [1]:
import sqlite3
from sqlite3 import Error

data_cache_prefix = 'data_cache/'

database_file = data_cache_prefix+"author.db"

## 2. The database functions.

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def create_author_table(conn):
    sql_create_authors_table = """
        CREATE TABLE IF NOT EXISTS authors (
            id integer PRIMARY KEY,
            idx text,
            n text NOT NULL,
            a text NOT NULL,
            pc integer ,
            cn integer,
            hi integer,
            pi REAL,
            upi REAL,
            t text
        );
        """
    create_table(conn, sql_create_authors_table)

def create_author(conn, author):
    sql = """
    INSERT INTO authors (idx,n,a,pc,cn,hi,pi,upi,t) VALUES (?,?,?,?,?,?,?,?,?)
    """
    cur = conn.cursor()
    cur.execute(sql, author)
    return cur.lastrowid

## 3. Put all authors to the database.

In [3]:
'''
#index 1
#n O. Willum
#a Res. Center for Microperipherik, Technische Univ. Berlin, Germany
#pc 1
#cn 0
#hi 0
#pi 0.0000
#upi 0.0000
#t new product;product group;active product;long product lifetime;old product;product generation;new technology;environmental benefit;environmental choice;environmental consequence
'''
def process_author_sqlite():
    with open('data/AMiner-Author.txt', 'r') as file:
        data = file.read().split('\n\n')

    database = data_cache_prefix+"author.db"
    conn = create_connection(database)

    # create tables
    if conn is None:
        # create projects table
        print("Error! cannot create the database connection.")
    else:
        with conn:
            create_author_table(conn)

            detail = ['index', 'n', 'a', 'pc', 'cn', 'hi', 'pi', 'upi', 't']
            total = len(data)
            for n, auth in enumerate(data):
                info = auth.split('\n')
                if info[0] is '':
                    break
                index = info[0].split(' ')[1]
                author = [0,0,0,0,0,0,0,0,0]
                for num,each in enumerate(detail):
                    author[num] = info[num].replace('#'+each+' ', '')
                create_author(conn, author)
                if n % 10000 == 0:
                    print("processed %d/%d"%(n,total))
    conn.close()

process_author_sqlite()

2.6.0
processed 0/1712434
processed 10000/1712434
processed 20000/1712434
processed 30000/1712434
processed 40000/1712434
processed 50000/1712434
processed 60000/1712434
processed 70000/1712434
processed 80000/1712434
processed 90000/1712434
processed 100000/1712434
processed 110000/1712434
processed 120000/1712434
processed 130000/1712434
processed 140000/1712434
processed 150000/1712434
processed 160000/1712434
processed 170000/1712434
processed 180000/1712434
processed 190000/1712434
processed 200000/1712434
processed 210000/1712434
processed 220000/1712434
processed 230000/1712434
processed 240000/1712434
processed 250000/1712434
processed 260000/1712434
processed 270000/1712434
processed 280000/1712434
processed 290000/1712434
processed 300000/1712434
processed 310000/1712434
processed 320000/1712434
processed 330000/1712434
processed 340000/1712434
processed 350000/1712434
processed 360000/1712434
processed 370000/1712434
processed 380000/1712434
processed 390000/1712434
processe

## 4. Query the database

In [4]:
def filter(keyword, by_type):
    with create_connection(database_file) as conn:
        rows = conn.cursor().execute(
            f"""
            SELECT idx FROM authors WHERE {by_type} LIKE '%'||?||'%'
            """
        ,[keyword]).fetchall()
        return [row[0] for row in rows]

filter('math','t')[:10]

2.6.0


['61', '121', '137', '200', '239', '326', '445', '545', '618', '665']

In [5]:
with create_connection(database_file) as conn:
    print('query by index')
    print(conn.cursor().execute("""
    SELECT * FROM authors WHERE idx=12345
    """).fetchall())
    print('query by tag')
    print(conn.cursor().execute("""
    SELECT * FROM authors WHERE t LIKE '%math%'
    """).fetchmany(10))
    print(conn.cursor().execute("SELECT * FROM authors WHERE %s LIKE '%%'||?||'%%'"%('t'),['math']).fetchmany(10))

2.6.0
query by index
[(12345, '12345', 'Brendan Johnson', '', 1, 7, 1, 1.0, 0.1429, 'Xen guest;Xen hypervisor;O Employing Xen;Xen LiveCD;Xen Project Leader VP;Xen Security Modules;Xen Utilizing Xen security;Xen Virtualization;Xen community;Xen implementation')]
query by tag
[(61, '61', 'Xinping Yan', '', 1, 0, 0, 0.0, 0.0, 'GP algorithm;F-W algorithm;FW algorithm;mathematic model;traffic assignment model;different engine operating mode;different location;different network structure;Charlotte network;multimode nonadditive path cost'), (121, '121', 'Qiaoge Liu', '', 1, 0, 0, 0.0, 0.0, 'service provider;modern service;traditional service;ServiceSwitching cost;Switching Cost;available parameter;dynamic process;important role;key factor;mathematical model'), (137, '137', 'Xiaona Yan', '', 1, 0, 0, 0.0, 0.0, 'capillary tube;Critical Flux;flow characteristic;mathematical model;network model;neural network;numerical computation method;numerical solution;refrigerant flow;theoretical model'), (2

> Explored shield, a NoSQL option for storing data, opt a very poor data compression and loading speed.

In [ ]:
'''
#index 1
#n O. Willum
#a Res. Center for Microperipherik, Technische Univ. Berlin, Germany
#pc 1
#cn 0
#hi 0
#pi 0.0000
#upi 0.0000
#t new product;product group;active product;long product lifetime;old product;product generation;new technology;environmental benefit;environmental choice;environmental consequence
'''
# def process_author_shelve():
#     with open('data/AMiner-Author.txt', 'r') as file:
#         data = file.read().split('\n\n')
#
#     with shelve.open('authors.shelve') as authors:
#         detail = ['n', 'a', 'pc', 'cn', 'hi', 'pi', 'upi', 't']
#         for auth in data:
#             info = auth.split('\n')
#             if info[0] is '':
#                 break
#             index = info[0].split(' ')[1]
#             author = {}
#             for num,each in enumerate(detail):
#                 author[each] = info[num+1].replace('#'+each+' ', '')
#             author['a'] = author['a'].split(', ')
#             author['t'] = author['t'].split(';')
#             author['field'] = []
#             author['field']+=([item for field in author['t'] for item in field.split(' ')])
#             authors[index] = author
#         return authors

# authors = process_author_shelve()